In [1]:
# import required libraries
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from stochastic_calculation import slowDcalculation
# stochastic calculation from ta module 
from ta.momentum import stoch

In [2]:
# create an object 
slowd = slowDcalculation(epic='CS.D.USCGC.TODAY.IP')
# login 
slowd.set_connection()

Connected successfully!


Fetch data from IG API

In [3]:
# load current file path 
root_path = os.getcwd()

# 1 min data file path (for resample)
existing_1min_data_path = os.path.join(root_path, 'data\gold_minutely_data.csv')
# 1 hour data file path
existing_1hr_data_path = os.path.join(root_path, 'data\gold_hourly_data.csv')
# 1 day data file path 
existing_1d_data_path = os.path.join(root_path, 'data\gold_daily_data.csv')

In [4]:
transformed_1m_df = slowd.fetch_gold_data(existing_1min_data_path, resolution="1Min", date_column = 'Date')
print("1 min", transformed_1m_df)

transformed_1hr_df = slowd.fetch_gold_data(existing_1hr_data_path, resolution="1h", date_column = 'Date')
print("1 hr", transformed_1hr_df)

transformed_1d_df = slowd.fetch_gold_data(existing_1d_data_path, resolution="1D", date_column = 'Date')
print("1 day", transformed_1d_df)

1 min                         Open     High      Low    Close  Volume
DateTime                                                       
2024-08-16 21:58:00  2507.53  2507.95  2507.38  2507.58      70
1 hr                         Open     High      Low    Close  Volume
DateTime                                                       
2024-08-16 21:00:00  2508.02  2509.68  2505.36  2507.58    4905
1 day               Open     High      Low    Close  Volume
DateTime                                             
2024-08-16  2454.8  2509.68  2450.55  2507.58  152451


Update data

In [5]:
# update dataset 
updated_1m_df = slowd.update_gold_data(existing_1min_data_path, transformed_1m_df)
# save to the file 
updated_1m_df.to_csv(existing_1min_data_path)

updated_1hr_df = slowd.update_gold_data(existing_1hr_data_path, transformed_1hr_df)
# save to the file 
updated_1hr_df.to_csv(existing_1hr_data_path)

updated_1d_df = slowd.update_gold_data(existing_1d_data_path, transformed_1d_df)
# save to the file 
updated_1d_df.to_csv(existing_1d_data_path)

Load the data

In [6]:
# load the data back 
minutely_data = pd.read_csv(existing_1min_data_path, parse_dates=['Date'], index_col='Date')
hourly_data = pd.read_csv(existing_1hr_data_path, parse_dates=['Date'], index_col='Date')
daily_data = pd.read_csv(existing_1d_data_path, parse_dates=['Date'], index_col='Date')

Resample into 3, 4, 6, and 9 hours

In [7]:
# check 4-hour resample 
df_4h = slowd.resample_data(minutely_data, '4h')
# print(df_4h.tail(30))
# calculate slowd of 4-hour resample 
df_4h_stochastic = slowd.stochastic(df_4h)
print(df_4h_stochastic.tail(5))
# check point
assert round(df_4h_stochastic.loc['2024-08-14 20:00:00']['SlowD'],4) == 27.2868
print('Test passed')

                        Open     High      Low    Close  Volume      FastK  \
Date                                                                         
2024-08-16 04:00:00  2454.89  2457.73  2450.55  2454.91   18305  60.232068   
2024-08-16 08:00:00  2454.98  2468.12  2451.07  2464.54   21877  85.627637   
2024-08-16 12:00:00  2464.56  2500.10  2464.56  2487.07   48452  80.846685   
2024-08-16 16:00:00  2487.10  2505.50  2485.62  2503.26   32869  96.949476   
2024-08-16 20:00:00  2503.24  2509.68  2501.60  2507.58   12475  97.294163   

                         FastD      SlowD  
Date                                       
2024-08-16 04:00:00  60.232068  57.201103  
2024-08-16 08:00:00  85.627637  71.414370  
2024-08-16 12:00:00  80.846685  76.130528  
2024-08-16 16:00:00  96.949476  86.540002  
2024-08-16 20:00:00  97.294163  91.917082  
Test passed


In [8]:
df_4h.loc["2024-08-01 04:00:00"]

Open       2451.290000
High       2451.450000
Low        2437.400000
Close      2446.990000
Volume    18263.000000
FastK        82.422177
FastD        82.422177
SlowD        61.916868
Name: 2024-08-01 04:00:00, dtype: float64

Stochastic calculation with ta module

In [9]:
# ta module stochastic calculation 
def calculate_stochastic(df, k_period=9, d_period=3):
    df['SlowK'] = stoch(df['High'], df['Low'], df['Close'], window=k_period, smooth_window=1)
    df['SlowD'] = df['SlowK'].rolling(window=d_period).mean()
    df.drop(columns = ['SlowK'], inplace=True)
    return df

In [10]:
df_4h_ta = calculate_stochastic(df_4h)
df_4h_ta.tail(5)

,Open,High,Low,Close,Volume,FastK,FastD,SlowD
Date,,,,,,,,
2024-08-16 04:00:00,2454.89,2457.73,2450.55,2454.91,18305,60.232068,60.232068,56.023830
2024-08-16 08:00:00,2454.98,2468.12,2451.07,2464.54,21877,85.627637,85.627637,68.706048
2024-08-16 12:00:00,2464.56,2500.10,2464.56,2487.07,48452,80.846685,80.846685,75.568797
2024-08-16 16:00:00,2487.10,2505.50,2485.62,2503.26,32869,96.949476,96.949476,87.807933
2024-08-16 20:00:00,2503.24,2509.68,2501.60,2507.58,12475,97.294163,97.294163,91.696775


Slow D values have spread than manual calculation. This is because the "stoch" method in ta module use simple moving average to smooth fast D. The d period smoothing is simple moving average. That's why the values have spreads. 

***Caution:*** Simple Moving Average (SMA) !!!

In [11]:
# resample 3, 6, 9 hours
df_3h =  slowd.resample_data(minutely_data, '3h')
df_6h = slowd.resample_data(minutely_data, '6h')
df_9h = slowd.resample_data(minutely_data, '9h')

In [12]:
# 3 hour slow D
df_3h_slowd = slowd.stochastic(df_3h)
df_3h_slowd.tail(5)

,Open,High,Low,Close,Volume,FastK,FastD,SlowD
Date,,,,,,,,
2024-08-16 09:00:00,2461.91,2468.12,2459.02,2464.54,15818,85.627637,85.627637,76.974717
2024-08-16 12:00:00,2464.56,2500.10,2464.56,2494.70,32940,92.062325,92.062325,84.518521
2024-08-16 15:00:00,2494.75,2497.75,2477.43,2495.88,33885,92.419616,92.419616,88.469069
2024-08-16 18:00:00,2495.90,2508.93,2492.98,2507.82,22066,98.098664,98.098664,93.283866
2024-08-16 21:00:00,2508.02,2509.68,2505.36,2507.58,4905,96.448503,96.448503,94.866185


In [13]:
# 6 hour slow D
df_6h_slowd = slowd.stochastic(df_6h)
df_6h_slowd.tail(5)

,Open,High,Low,Close,Volume,FastK,FastD,SlowD
Date,,,,,,,,
2024-08-15 18:00:00,2455.71,2461.60,2453.40,2454.69,25541,47.580984,47.580984,46.799519
2024-08-16 00:00:00,2454.80,2459.98,2450.55,2452.64,26390,43.268826,43.268826,45.034173
2024-08-16 06:00:00,2452.58,2468.12,2451.07,2464.54,32265,68.300379,68.300379,56.667276
2024-08-16 12:00:00,2464.56,2500.10,2464.56,2495.88,66825,93.796854,93.796854,75.232065
2024-08-16 18:00:00,2495.90,2509.68,2492.98,2507.58,26971,97.294163,97.294163,86.263114


In [14]:
def determine_market_cycle(long_time_frame: pd.DataFrame, short_time_frame: pd.DataFrame) -> str:
    """"
    A function that decides wheter a market is up bias, down bias or neutral
    by comparing long time frame and short time frame slow d values.
    
    Parameters
    ----------
    long_time_frame: longer time frame
    short_time_frame: short time frame
    """

    long_time_frame_current_D = round(long_time_frame['SlowD'][-1], 4)
    long_time_frame_prev_D = round(long_time_frame['SlowD'][-2], 4)

    print("Current Slow D (long): ", long_time_frame_current_D)
    print("Previous Slow D (long): ", long_time_frame_prev_D)

    short_time_frame_current_D = round(short_time_frame['SlowD'][-1], 4)
    short_time_frame_prev_D = round(short_time_frame['SlowD'][-2], 4)

    print("Current Slow D (short): ", short_time_frame_current_D)
    print("Previous Slow D (short): ", short_time_frame_prev_D)

    if long_time_frame_current_D > long_time_frame_prev_D and short_time_frame_current_D > short_time_frame_prev_D:
        return "Up Bias"
    elif long_time_frame_current_D < long_time_frame_prev_D and short_time_frame_current_D < short_time_frame_prev_D:
        return "Down Bias"
    else:
        return "Neutral"

In [15]:
print(f"Current 6 hour Slow D: {round(df_6h_slowd['SlowD'][-1], 4)}")
print(f"Previous 6 hour Slow D: {round(df_6h_slowd['SlowD'][-2], 4)}")
print(f"Current 3 hour Slow D: {round(df_3h_slowd['SlowD'][-1], 4)}")
print(f"Previous 3 hour Slow D: {round(df_3h_slowd['SlowD'][-2], 4)}")

Current 6 hour Slow D: 86.2631
Previous 6 hour Slow D: 75.2321
Current 3 hour Slow D: 94.8662
Previous 3 hour Slow D: 93.2839


In [16]:
marketcycle_6hr = determine_market_cycle(df_6h_slowd, df_3h_slowd)
print(f"6 hr market cycle: {marketcycle_6hr}.")

Current Slow D (long):  86.2631
Previous Slow D (long):  75.2321
Current Slow D (short):  94.8662
Previous Slow D (short):  93.2839
6 hr market cycle: Up Bias.


Test with provided morning call from GSC 

GSC Morning Call for 1 August 2024

1 August 2024, Thursday. Good Morning!			
				
TaelSG prices now 3967			
May move between 3901 and 3977 during Asian hours today.
				
XAUSGD prices now 3271			
May move between 3210 and 3273 during Asian hours today.
				
Market mode: Cyclical	
		
6 Hr Market Cycle: Up Bias

First, extract 6 hr market cycles. 

In [17]:
aug_1_0hr_slowd = round(df_6h.loc['2024-08-1 00:00:00']['SlowD'], 4)
print("0 hr slow D", aug_1_0hr_slowd)
aug_1_6_hr_slowd = round(df_6h.loc['2024-08-1 06:00:00']['SlowD'], 4)
print("6 hr slow D", aug_1_6_hr_slowd)

0 hr slow D 35.0628
6 hr slow D 41.7027


Now, extract 3 hr market cycles. 

In [18]:
aug_1_0hr_slowd = round(df_3h.loc['2024-08-1 00:00:00']['SlowD'], 4)
print("0 hr slow D", aug_1_0hr_slowd)
aug_1_3_hr_slowd = round(df_3h.loc['2024-08-1 03:00:00']['SlowD'], 4)
print("3 hr slow D", aug_1_3_hr_slowd)
aug_1_6_hr_slowd = round(df_3h.loc['2024-08-1 06:00:00']['SlowD'], 4)
print("6 hr slow D", aug_1_6_hr_slowd)

0 hr slow D 52.4773
3 hr slow D 64.1742
6 hr slow D 71.6334


According to this data comparison, the market is "Up Bias". I remove slow D of 12:00 at 6 hour interval period and slow D of 9:00 at 3 hour interval. Because in reality, the current time is at 7:22 and those interval are not completed yet. Therfore, I remove them for now. 

Let's segment out the last minutes that GSC provided us. 

In [19]:
aug1_segment = minutely_data.loc["2024-04-25 18:16:00":"2024-08-1 7:22:00"]
df_6h_resample = slowd.resample_data(aug1_segment, '6h')
df_3h_resample = slowd.resample_data(aug1_segment, '3h')
# calculate slow d
stoch_df_6h = slowd.stochastic(df_6h_resample)
stoch_df_3h = slowd.stochastic(df_3h_resample)

In [20]:
aug_1_0hr_slowd = round(stoch_df_6h.loc['2024-08-1 00:00:00']['SlowD'], 4)
print("0 hr slow D", aug_1_0hr_slowd)
aug_1_6_hr_slowd = round(stoch_df_6h.loc['2024-08-1 06:00:00']['SlowD'], 4)
print("6 hr slow D", aug_1_6_hr_slowd)

0 hr slow D 35.0628
6 hr slow D 50.686


In [21]:
aug_1_0hr_slowd = round(stoch_df_3h.loc['2024-08-1 00:00:00']['SlowD'], 4)
print("0 hr slow D", aug_1_0hr_slowd)
aug_1_3_hr_slowd = round(stoch_df_3h.loc['2024-08-1 03:00:00']['SlowD'], 4)
print("3 hr slow D", aug_1_3_hr_slowd)
aug_1_6_hr_slowd = round(stoch_df_3h.loc['2024-08-1 06:00:00']['SlowD'], 4)
print("6 hr slow D", aug_1_6_hr_slowd)

0 hr slow D 52.4773
3 hr slow D 64.1742
6 hr slow D 73.7551


In [22]:
marketcycle_6hr = determine_market_cycle(stoch_df_6h, stoch_df_3h)
print(f"6 hr market cycle: {marketcycle_6hr}.")

Current Slow D (long):  50.686
Previous Slow D (long):  35.0628
Current Slow D (short):  73.7551
Previous Slow D (short):  64.1742
6 hr market cycle: Up Bias.


2 August 2024, Friday. Good Morning!			

TaelSG prices now 3959			
May move between 3949 and 4001 during Asian hours today.
				
XAUSGD prices now 3265			
May move between 3249 and 3292 during Asian hours today.
						
Market mode: Trending Up	
		
6 Hr Market Cycle: Down Bias

In [23]:
aug2_segment = minutely_data.loc["2024-04-25 18:16:00":"2024-08-2 5:00:00"]
df_6h_resample = slowd.resample_data(aug2_segment, '6h')
df_3h_resample = slowd.resample_data(aug2_segment, '3h')
# calculate slow d
stoch_df_6h = slowd.stochastic(df_6h_resample)
stoch_df_3h = slowd.stochastic(df_3h_resample)
marketcycle_6hr = determine_market_cycle(stoch_df_6h, stoch_df_3h)
print(f"6 hr market cycle: {marketcycle_6hr}.")

Current Slow D (long):  78.3472
Previous Slow D (long):  65.695
Current Slow D (short):  73.4593
Previous Slow D (short):  66.1745
6 hr market cycle: Up Bias.


14 August 2024, Wednesday. Good Morning!			
				
TaelSG prices now 3942 per tael		
May move between 3936 and 3965 during Asian hours today.
				
XAUSGD prices now 3250 per oz		
May move between 3238 and 3263 during Asian hours today.
				
Market mode: Trending Up	
		
6 Hr Market Cycle: Up Bias

In [24]:
aug16_segment = minutely_data.loc["2024-04-25 18:16:00":"2024-08-16 7:29:00"]
df_6h_resample = slowd.resample_data(aug16_segment, '6h')
df_3h_resample = slowd.resample_data(aug16_segment, '3h')
# calculate slow d
stoch_df_6h = slowd.stochastic(df_6h_resample)
stoch_df_3h = slowd.stochastic(df_3h_resample)

In [25]:
marketcycle_6hr = determine_market_cycle(stoch_df_6h, stoch_df_3h)
print(f"6 hr market cycle: {marketcycle_6hr}.")

Current Slow D (long):  48.8318
Previous Slow D (long):  45.0342
Current Slow D (short):  61.9927
Previous Slow D (short):  58.0044
6 hr market cycle: Up Bias.


Three out of two correct. May be need to work on live slow D.

In [26]:
""" def resample(df: pd.DataFrame, interval: str) -> pd.DataFrame:
    df.index = pd.to_datetime(df.index)
    
    # Resample the dataset to the specified interval
    df_resampled = df.resample(interval).agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    }).dropna()
    print("Resample:", df_resampled)

    # Handle the ongoing interval (e.g., from 06:01 to 07:22)
    df_ongoing = df[df.index > df_resampled.index[-1]]
    print("Ongoing:", df_ongoing)
    
    if not df_ongoing.empty:
        open_ = df_ongoing['Open'].iloc[0]
        high_ = df_ongoing['High'].max()
        low_ = df_ongoing['Low'].min()
        close_ = df_ongoing['Close'].iloc[-1]
        volume_ = df_ongoing['Volume'].sum()
        
        new_row = pd.DataFrame({
            'Open': [open_],
            'High': [high_],
            'Low': [low_],
            'Close': [close_],
            'Volume': [volume_]
        }, index=[df_ongoing.index[-1]])
        
        df_resampled = pd.concat([df_resampled, new_row])


    return df_resampled """

' def resample(df: pd.DataFrame, interval: str) -> pd.DataFrame:\n    df.index = pd.to_datetime(df.index)\n    \n    # Resample the dataset to the specified interval\n    df_resampled = df.resample(interval).agg({\n        \'Open\': \'first\',\n        \'High\': \'max\',\n        \'Low\': \'min\',\n        \'Close\': \'last\',\n        \'Volume\': \'sum\'\n    }).dropna()\n    print("Resample:", df_resampled)\n\n    # Handle the ongoing interval (e.g., from 06:01 to 07:22)\n    df_ongoing = df[df.index > df_resampled.index[-1]]\n    print("Ongoing:", df_ongoing)\n    \n    if not df_ongoing.empty:\n        open_ = df_ongoing[\'Open\'].iloc[0]\n        high_ = df_ongoing[\'High\'].max()\n        low_ = df_ongoing[\'Low\'].min()\n        close_ = df_ongoing[\'Close\'].iloc[-1]\n        volume_ = df_ongoing[\'Volume\'].sum()\n        \n        new_row = pd.DataFrame({\n            \'Open\': [open_],\n            \'High\': [high_],\n            \'Low\': [low_],\n            \'Close\': [clo

Test segment 

In [34]:
test_segment = minutely_data.loc["2024-04-25 18:16:00":"2024-08-02 7:35:00"]
test_segment_6h_live = slowd.resample_data(test_segment, '6h')
test_segment_6h_slowd = slowd.stochastic(test_segment_6h_live)
test_segment_6h_slowd.tail(5)

,Open,High,Low,Close,Volume,FastK,FastD,SlowD
Date,,,,,,,,
2024-08-01 06:00:00,2442.72,2448.84,2430.22,2433.00,28833,48.342575,48.342575,41.702684
2024-08-01 12:00:00,2433.01,2462.21,2432.01,2445.14,33154,67.512157,67.512157,54.607420
2024-08-01 18:00:00,2445.19,2448.72,2434.96,2446.32,20365,76.782583,76.782583,65.695002
2024-08-02 00:00:00,2447.16,2459.46,2434.69,2454.50,23298,88.734658,88.734658,77.214830
2024-08-02 06:00:00,2454.45,2461.37,2454.30,2459.66,5968,96.274109,96.274109,86.744469


In [35]:
test_segment_3h_live = slowd.resample_data(test_segment, '3h')
test_segment_3h_slowd = slowd.stochastic(test_segment_3h_live)
test_segment_3h_slowd.tail(5)

,Open,High,Low,Close,Volume,FastK,FastD,SlowD
Date,,,,,,,,
2024-08-01 18:00:00,2445.19,2448.72,2434.96,2439.59,14538,65.990077,65.990077,71.051083
2024-08-01 21:00:00,2439.60,2446.50,2438.31,2446.32,5827,75.610130,75.610130,73.330607
2024-08-02 00:00:00,2447.16,2452.31,2443.64,2449.10,8526,59.018443,59.018443,66.174525
2024-08-02 03:00:00,2449.09,2459.46,2434.69,2454.50,14772,75.898718,75.898718,71.036622
2024-08-02 06:00:00,2454.45,2461.37,2454.30,2459.66,5968,92.028759,92.028759,81.532690


In [29]:
sixhr_segment = test_segment.loc["2024-08-02 06:00:00":"2024-08-02 07:35:00"]
print(sixhr_segment)
open = sixhr_segment['Open'].iloc[0]
high = sixhr_segment['High'].max()
low = sixhr_segment['Low'].min()
close = sixhr_segment['Close'].iloc[-1]
volume = sixhr_segment['Volume'].sum()
print("Open: ", open)
print("High: ", high)
print("Low: ", low)
print("Close: ", close)
print("Volume: ", volume)

                        Open     High      Low    Close  Volume
Date                                                           
2024-08-02 06:00:00  2454.45  2454.65  2454.30  2454.49      68
2024-08-02 06:01:00  2454.48  2454.90  2454.36  2454.63      57
2024-08-02 06:02:00  2454.64  2454.98  2454.46  2454.74      53
2024-08-02 06:03:00  2454.78  2455.08  2454.51  2454.70      51
2024-08-02 06:04:00  2454.70  2455.34  2454.65  2455.30      58
...                      ...      ...      ...      ...     ...
2024-08-02 07:31:00  2460.63  2460.70  2459.76  2459.99      89
2024-08-02 07:32:00  2459.94  2459.94  2459.20  2459.41      82
2024-08-02 07:33:00  2459.39  2459.70  2458.69  2458.69      82
2024-08-02 07:34:00  2458.70  2460.36  2458.67  2460.11      86
2024-08-02 07:35:00  2460.12  2460.72  2459.61  2459.66      86

[96 rows x 5 columns]
Open:  2454.45
High:  2461.37
Low:  2454.3
Close:  2459.66
Volume:  5968


In [30]:
sixhr_slowd = slowd.stochastic(sixhr_segment)
sixhr_slowd.tail(5)

,Open,High,Low,Close,Volume,FastK,FastD,SlowD
Date,,,,,,,,
2024-08-02 07:31:00,2460.63,2460.70,2459.76,2459.99,89,62.191781,62.191781,66.288839
2024-08-02 07:32:00,2459.94,2459.94,2459.20,2459.41,82,44.943820,44.943820,55.616330
2024-08-02 07:33:00,2459.39,2459.70,2458.69,2458.69,82,18.292683,18.292683,36.954506
2024-08-02 07:34:00,2458.70,2460.36,2458.67,2460.11,86,61.585366,61.585366,49.269936
2024-08-02 07:35:00,2460.12,2460.72,2459.61,2459.66,86,47.865854,47.865854,48.567895


In [31]:
test_segment_6h_slowd.tail(5)

,Open,High,Low,Close,Volume,FastK,FastD,SlowD
Date,,,,,,,,
2024-08-01 06:00:00,2442.72,2448.84,2430.22,2433.00,28833,48.342575,48.342575,41.702684
2024-08-01 12:00:00,2433.01,2462.21,2432.01,2445.14,33154,67.512157,67.512157,54.607420
2024-08-01 18:00:00,2445.19,2448.72,2434.96,2446.32,20365,76.782583,76.782583,65.695002
2024-08-02 00:00:00,2447.16,2459.46,2434.69,2454.50,23298,88.734658,88.734658,77.214830
2024-08-02 06:00:00,2454.45,2461.37,2454.30,2459.66,5968,96.274109,96.274109,86.744469


In [32]:
test_segment_3h_slowd.tail(5)

,Open,High,Low,Close,Volume,FastK,FastD,SlowD
Date,,,,,,,,
2024-08-01 18:00:00,2445.19,2448.72,2434.96,2439.59,14538,65.990077,65.990077,71.051083
2024-08-01 21:00:00,2439.60,2446.50,2438.31,2446.32,5827,75.610130,75.610130,73.330607
2024-08-02 00:00:00,2447.16,2452.31,2443.64,2449.10,8526,59.018443,59.018443,66.174525
2024-08-02 03:00:00,2449.09,2459.46,2434.69,2454.50,14772,75.898718,75.898718,71.036622
2024-08-02 06:00:00,2454.45,2461.37,2454.30,2459.66,5968,92.028759,92.028759,81.532690


In [37]:
test_segment_4h_live = slowd.resample_data(test_segment, '4h')
test_segment_4h_slowd = slowd.stochastic(test_segment_4h_live)
test_segment_4h_slowd.tail(5)

,Open,High,Low,Close,Volume,FastK,FastD,SlowD
Date,,,,,,,,
2024-08-01 12:00:00,2433.01,2457.38,2432.01,2453.28,20782,92.163543,92.163543,76.749931
2024-08-01 16:00:00,2456.56,2462.21,2435.80,2439.38,22470,66.642314,66.642314,71.696123
2024-08-01 20:00:00,2439.32,2446.50,2434.96,2446.32,10267,76.782583,76.782583,74.239353
2024-08-02 00:00:00,2447.16,2452.31,2434.69,2447.14,13841,77.341753,77.341753,75.790553
2024-08-02 04:00:00,2447.15,2461.37,2446.80,2459.66,15425,96.085955,96.085955,85.938254
